In [30]:
# !pip install -e .

In [1]:
from deeplens.extractor import ExtractSingleSample
from deeplens.intervene import InterveneFeatures, ReinjectSingleSample
from deeplens.utils.analysis import plot_topk_distribution, get_top_k_tokens

In [ ]:
def pipeline(
        text,
        model: str = "",
        layer: int = 3,
        sae_model: str = r"saved_models\gpt2L3-untied\run_20251229_102023\best_model.pt",
        sae_config: str = r"C:\code\deeplens\demo\config.yaml",
        feature: int = -1,
        alpha: float = 5.0,
        tok_position: int = -1,
        generate: bool = False,
        max_new_tokens: int = 25,
        temperature: float = 1.0
    ):
    extractor = ExtractSingleSample(
        model=model,
        layer=layer,
        max_length=1024,
        device="auto"
    )

    intervene = InterveneFeatures(
        sae_model=sae_model,
        sae_config=sae_config,
        device="auto"
    )

    acts = extractor.get_mlp_acts(text)
    features = intervene.get_alive_features(acts, token_position=-1)

    original, mod = intervene.intervene_feature(
        activations=acts,
        feature=features[feature].item(),
        alpha=alpha,
        token_positions=tok_position
    )
    reinject = ReinjectSingleSample(
        hf_model=model,
        device="auto"
    )
    modified = reinject.reinject_and_generate(
        text=text,
        modified_activations=mod,
        layer=layer,
        generate=generate,
        max_new_tokens=max_new_tokens,
        temperature=temperature
    )
    return modified, original